In [1]:
vocab = ["<pad>", 'El', 'perro', 'persigue', 'al', 'gato', 'y', 'se', 'lo', 'come', "<end>"]

def text_to_tokens(text, vocab):
    tokens = text.split(" ")
    return [vocab.index(token) if token in vocab else vocab.index("<pad>") for token in tokens]

frase = "El perro persigue al gato"
tokens = text_to_tokens(frase, vocab)
print(f"Tokens: {tokens}")

Tokens: [1, 2, 3, 4, 5]


In [2]:
#from transformer import *
import torch
from Transformer import TransformerEncoder, TransformerDecoder

# covertimos a un tensor los tokens

In [3]:
tokens = torch.tensor(tokens, dtype=torch.long)
tokens

tensor([1, 2, 3, 4, 5])

# Parametros

In [ ]:
vocab_size = len(vocab)
d_model = 512
num_layers = 6
num_heads = 8
d_ff = 2048
dropout = 0.1
max_seq_length = 5 # este tiene que tener el mismo tamaño de los tokens (checar tokens.shape[1] si es de una dimension, tokens.shape[1] si son dos dimensiones)
batch_size = 1

src_mask = torch.ones(batch_size, 1, max_seq_length, max_seq_length) # mascara que es una matriz de solos 1s porque no tenemos tokens con padding

In [8]:
tokens = tokens.unsqueeze(0)

In [6]:
tokens.shape

torch.Size([1, 5])

In [9]:
tokens.size(1)

5

# Transformer Encoder

In [20]:
encoder = TransformerEncoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_seq_length)
output_encoder = encoder(tokens, src_mask)

In [21]:
output_encoder.shape

torch.Size([1, 5, 512])

In [22]:
output_encoder

tensor([[[-1.4020,  0.3544,  1.4220,  ...,  1.3007, -0.2529,  1.6627],
         [-0.5963, -0.0571, -1.3349,  ..., -0.9605, -0.5553,  0.6529],
         [-0.8698,  0.4888, -0.5234,  ...,  2.2267,  0.4915,  0.3641],
         [-0.4291,  0.1886, -0.6805,  ...,  1.6035,  1.0038,  0.3653],
         [-1.7163,  0.9961, -0.1121,  ..., -2.2682,  0.3475, -0.3644]]],
       grad_fn=<NativeLayerNormBackward0>)

# Transformer Decoder

In [23]:
tgt = torch.tril(torch.ones(4, 4))
print(tgt)
print(tgt.shape)

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
torch.Size([4, 4])


In [24]:
x = tokens[:, :-1]
print(x)

tensor([[1, 2, 3, 4]])


In [25]:
decoder = TransformerDecoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_seq_length)
output_decoder = decoder(x, output_encoder, tgt)

In [26]:
print(output_decoder)

tensor([[[-1.0115,  0.0571,  1.8621,  ..., -1.8423,  2.1023,  0.9634],
         [ 0.6030,  0.0345, -0.1469,  ..., -0.0258,  0.7944,  1.8433],
         [ 0.6364, -1.8943,  0.4000,  ..., -0.3813,  2.7131,  0.8408],
         [-0.7746, -0.8679,  0.1132,  ..., -1.3419,  1.8634,  2.0326]]],
       grad_fn=<NativeLayerNormBackward0>)


adding a dense layer

In [27]:
layer = torch.nn.Linear(d_model, vocab_size)
salida_probabilistica = layer(output_decoder)
print(salida_probabilistica)

tensor([[[-1.6646,  0.7465, -0.7856,  0.2578,  0.3722, -0.8806,  0.1744,
          -0.6289,  0.3803, -0.2897,  0.7315],
         [ 0.2475, -0.0300, -0.1901,  0.7969,  0.2784, -0.4264, -0.0627,
           0.0933,  0.5479, -0.2422, -0.0995],
         [-0.5493,  0.9710, -0.3939, -0.1515, -0.0917, -0.6521, -0.1032,
          -0.9892,  0.4811, -0.3036,  0.0455],
         [-0.6274,  0.4968,  0.1906, -0.4895,  0.2001, -1.2997, -0.0390,
           1.0698,  0.5241,  0.4972,  1.1193]]], grad_fn=<ViewBackward0>)


In [28]:
print(salida_probabilistica.shape)

torch.Size([1, 4, 11])


In [29]:
probabilidades = torch.softmax(salida_probabilistica, dim=-1)
print(probabilidades.shape)
print(probabilidades)

torch.Size([1, 4, 11])
tensor([[[0.0159, 0.1769, 0.0382, 0.1085, 0.1216, 0.0348, 0.0998, 0.0447,
          0.1226, 0.0628, 0.1742],
         [0.1006, 0.0762, 0.0650, 0.1743, 0.1038, 0.0513, 0.0738, 0.0863,
          0.1359, 0.0617, 0.0711],
         [0.0532, 0.2431, 0.0621, 0.0791, 0.0840, 0.0480, 0.0830, 0.0342,
          0.1489, 0.0680, 0.0964],
         [0.0339, 0.1042, 0.0767, 0.0389, 0.0775, 0.0173, 0.0610, 0.1849,
          0.1071, 0.1043, 0.1943]]], grad_fn=<SoftmaxBackward0>)


In [30]:
predicciones = torch.argmax(probabilidades, dim=-1)
print(predicciones.shape)
print(predicciones)

torch.Size([1, 4])
tensor([[ 1,  3,  1, 10]])


In [31]:
secuencia_generada = [vocab[idx] for idx in predicciones[0].tolist()]
print("Secuencia generada: ", secuencia_generada)

Secuencia generada:  ['El', 'persigue', 'El', '<end>']
